In [ ]:
import pynetdicom
import pydicom

import copy

from pymedphys._dicom.ct import extend

In [ ]:
INCOMING_PORT = 33304
# TPS_PORT = 35104

# TPS_IP = '127.0.0.1'

TPS_PORT = 104
TPS_IP = 'rccc-monacoda'

In [ ]:
pynetdicom.debug_logger()

In [ ]:
def extend_datasets(datasets, number_of_slices=20):
    deque_datasets = extend.convert_datasets_to_deque(datasets)
    extend.extend_datasets(deque_datasets, 0, number_of_slices)
    extend.extend_datasets(deque_datasets, -1, number_of_slices)
    
    return deque_datasets

In [ ]:
def send_datasets(datasets, ae):
    ae.add_requested_context(pynetdicom.sop_class.CTImageStorage)

    assoc = ae.associate(TPS_IP, TPS_PORT)
    if assoc.is_established:
        for ds in datasets:
            ds.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
            ds.fix_meta_info(enforce_standard=True)
            status = assoc.send_c_store(ds)
            
        assoc.release()

In [ ]:
# debugging

# datasets = list(dicom_store.values())[0]
# extended = extend_datasets(datasets)

# send_datasets(extended, ae)

In [ ]:
dicom_store = {}


def handle_open(event):
    return 0x0000


def handle_accepted(event): 
    dicom_store[event.assoc] = []
    
    return 0x0000


def handle_store(event):
    dicom_store[event.assoc].append(event.dataset)
    
    return 0x0000


def handle_release(event, ae):
    datasets = extend_datasets(dicom_store[event.assoc])
    send_datasets(datasets, ae)
    
    return 0x0000

In [ ]:
ae = pynetdicom.AE()

handlers = [
    (pynetdicom.evt.EVT_CONN_OPEN, handle_open),
    (pynetdicom.evt.EVT_ACCEPTED, handle_accepted),
    (pynetdicom.evt.EVT_C_STORE, handle_store), 
    (pynetdicom.evt.EVT_RELEASED, handle_release, [ae])
]

ae.network_timeout = None
ae.acse_timeout = None
ae.dimse_timeout = None
ae.maximum_pdu_size = 0

storage_sop_classes = [cx.abstract_syntax for cx in pynetdicom.AllStoragePresentationContexts]
for uid in storage_sop_classes:
    ae.add_supported_context(uid, pynetdicom.ALL_TRANSFER_SYNTAXES)
    
ae.start_server(("", INCOMING_PORT), block=True, evt_handlers=handlers)